# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv

## Import required libraries and environment variables

In [1]:
import os
import json
import openai
import sys

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SearchField,
    SemanticSettings,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
)
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [3]:
# Configure environment variables
load_dotenv("azure.env")

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = "quickdemo"
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

credential = AzureKeyCredential(key)

## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [4]:
!ls text-sample.json -lh

-rwxrwxrwx 1 root root 74K Aug 23 14:01 text-sample.json


In [5]:
# Generate Document Embeddings using OpenAI Ada 002

# Read the text-sample.json
with open("text-sample.json", "r", encoding="utf-8") as file:
    input_data = json.load(file)


@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(input=text, engine="text-embedding-ada-002")
    embeddings = response["data"][0]["embedding"]
    return embeddings


# Generate embeddings for title and content fields
for item in input_data:
    title = item["title"]
    content = item["content"]
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item["titleVector"] = title_embeddings
    item["contentVector"] = content_embeddings

# Output embeddings to docVectors.json file
with open("docVectors.json", "w") as f:
    json.dump(input_data, f)

## Create your search index
Create your search index schema and vector search configuration:

In [6]:
# Create a search index
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
        facetable=True,
    ),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(
        name="titleVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_configuration="my-vector-config",
    ),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_configuration="my-vector-config",
    ),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine",
            },
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")],
    ),
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_settings=semantic_settings,
)
result = index_client.create_or_update_index(index)
print(f" {result.name} created")

 quickdemo created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [7]:
# Upload some documents to the index
with open("docVectors.json", "r") as file:
    documents = json.load(file)
search_client = SearchClient(
    endpoint=service_endpoint, index_name=index_name, credential=credential
)
result = search_client.upload_documents(documents)

print(f"Uploaded {len(documents)} documents")

Uploaded 108 documents


## Perform a vector similarity search

In [8]:
# Pure Vector Search
query = "I want to monitor my applications"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vectors=[vector],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure Application Insights
Score: 0.84312654
Content: Azure Application Insights is an application performance management service that enables you to monitor, diagnose, and troubleshoot your applications and infrastructure. It provides features like real-time telemetry, dependency mapping, and live metrics. Application Insights supports various platforms, such as .NET, Java, Node.js, and Python. You can use Azure Application Insights to detect and diagnose issues, optimize your performance, and ensure the availability of your applications. It also integrates with other Azure services, such as Azure Monitor and Azure Log Analytics.
Category: Management + Governance

Title: Azure Mobile Apps
Score: 0.81063944
Content: Azure Mobile Apps is a mobile app development platform that enables you to build, test, deploy, and monitor your mobile applications. It provides features like offline data sync, push notifications, and user authentication. Mobile Apps supports various platforms, inc

In [9]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)
query = "tools voor softwareontwikkeling"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vectors=[vector],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.80402035
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.79679435
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

## Perform a Cross-Field Vector Search

> We can search on the title vector as well

In [10]:
# Cross-Field Vector Search
query = "I want to monitor my applications"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(
    value=generate_embeddings(query), k=3, fields="titleVector, contentVector"
)

results = search_client.search(
    search_text=None,
    vectors=[vector],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure Monitor
Score: 0.03306011110544205
Content: Azure Monitor is a comprehensive, full-stack monitoring service that enables you to collect, analyze, and act on telemetry data from your applications and infrastructure. It provides features like log analytics, metrics, alerts, and dashboards. Azure Monitor supports various data sources, such as Azure resources, custom applications, and operating systems. You can use Azure Monitor to detect and diagnose issues, optimize your performance, and ensure the availability of your resources. It also integrates with other Azure services, such as Azure Log Analytics and Azure Application Insights.
Category: Management + Governance

Title: Azure Application Insights
Score: 0.03306011110544205
Content: Azure Application Insights is an application performance management service that enables you to monitor, diagnose, and troubleshoot your applications and infrastructure. It provides features like real-time telemetry, dependency mapping, and l

## Perform a Multi-Vector Search

In [11]:
# Multi-Vector Search
query = "tools for software development"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector1 = Vector(value=generate_embeddings(query), k=3, fields="titleVector")
vector2 = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vectors=[vector1, vector2],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var

## Perform a Pure Vector Search with a filter

In [12]:
# Pure Vector Search with Filter
query = "tools for software development"

search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vectors=[vector],
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.82971567
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81866795
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

## Perform a Hybrid Search

> adding the query to the search_text

In [13]:
# Hybrid Search
query = "Storage"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(
    search_text=query, vectors=[vector], select=["title", "content", "category"], top=3
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure Storage
Score: 0.03306011110544205
Content: Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and management options, including REST APIs, SDKs, and Azure Portal. You can secure your data using encryption at rest and in transit.
Category: Storage

Title: Azure Blob Storage
Score: 0.03201844170689583
Content: Azure Blob Storage is a scalable, durable, and high-performance object storage service for unstructured data. It provides features like data redundancy, geo-replication, and fine-grained access control. Blob Storage supports various data types, such as images, documents, and videos. You can use Blob Storage to store and manage your data, build data lakes, and 